<a href="https://colab.research.google.com/github/sabahonarmand/DataMining-Apriori/blob/main/Project1_Data_Mining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from itertools import chain, combinations
from collections import defaultdict
import pandas as pd
import numpy as np
import matplotlib
from mlxtend.preprocessing import TransactionEncoder
import matplotlib.pyplot as plt
from collections import defaultdict

freqSet = defaultdict(int)
largeSet = dict()

#read data from csv
dataframe = pd.read_csv("groceries.csv", header=None)

records = []

# for i in range (0, 9835):
#     records.append([str(dataframe.values[i,j]) for j in range(0, 32)])

TE = TransactionEncoder()

#True and False Table
array = TE.fit(records).transform(records)
transf_df = pd.DataFrame(array, columns = TE.columns_)


#drop nan column
basket = transf_df.drop(['nan'], axis = 1)


def get_transactionsList(fname):
    record = [] 
    with open(fname, "rU") as file_iter:
        for line in file_iter:
            line = line.strip().rstrip(",")  # Remove trailing comma
            record.append(frozenset(line.split(",")))
        return record

def get_items_collection(items):
    columns_name =[]
    for col in items.columns:
      columns_name.append(col)
    itemSet = set()
    for item in columns_name:
        itemSet.add(frozenset([item]))
    return itemSet    

items_collection = get_items_collection(basket)
transactionList = get_transactionsList("groceries.csv")


def returnItemsWithMinSupport(itemSet, transactionList, minSupport, freqSet):
    _itemSet = set()
    localSet = defaultdict(int)

    for item in itemSet:
        for transaction in transactionList:
            if item.issubset(transaction):
                freqSet[item] += 1
                localSet[item] += 1

    for item, count in localSet.items():
        support = float(count) / len(transactionList)

        if support >= minSupport:
            _itemSet.add(item)

    return _itemSet

def joinSet(itemSet, length):
    """Join a set with itself and returns the n-element itemsets"""
    return set(
        [i.union(j) for i in itemSet for j in itemSet if len(i.union(j)) == length]
    )

def getSupport(item):
  """local function which Returns the support of an item"""
  return float(freqSet[item]) / len(transactionList)
def subsets(arr):
    """ Returns non empty subsets of arr"""
    return chain(*[combinations(arr, i + 1) for i, a in enumerate(arr)])

class Arules():

  def get_frequent_item_sets(self,
  transactions,
  min_support):
      oneCSet = returnItemsWithMinSupport(items_collection, transactionList, 0.005, freqSet)
      currentLSet = oneCSet
      k = 2
      while len(currentLSet) != 0 :
        largeSet[k - 1] = currentLSet
        currentLSet = joinSet(currentLSet, k)
        currentCSet = returnItemsWithMinSupport(currentLSet, transactionList, 0.005, freqSet)
        currentLSet = currentCSet
        k += 1
      toRetItems = []
      for key, value in largeSet.items():
        toRetItems.extend([(tuple(item), getSupport(item)) for item in value])
      return toRetItems

      
  # def get_arules(self,transactionList,min_confidence=None,min_lift=None,sort_by='lift'):
  #   toRetRules = []
  #   for key, value in list(largeSet.items())[1:]:
  #     for item in value:
  #         _subsets = map(frozenset, [x for x in subsets(item)])
  #         for element in _subsets:
  #           remain = item.difference(element)
  #           if len(remain) > 0:
  #             confidence = getSupport(item,transactionList) / getSupport(element,transactionList)
  #             if confidence >= min_confidence:
  #               toRetRules.append(((tuple(element), tuple(remain)), confidence))
  #   return toRetRules
apriori = Arules()
result = apriori.get_frequent_item_sets(transactionList,0.005)
res = result[-10:]
# rule = apriori.get_arules(transactionList,0.005,0.5)

AttributeError: ignored